In [27]:
import pandas as pd
import numpy
import pylab
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import LinearRegression

ImportError: cannot import name LinearRegression

In [28]:
trainBodies = pd.read_csv('fncData/train_bodies.csv')
trainStances = pd.read_csv('fncData/train_stances.csv')
trainStancesRandom = pd.read_csv('fncData/train_stances.random.csv')

print "Train Bodies (20/%s):" % (len(trainBodies))
#print trainBodies.head(20)

# to get articleBody:
# trainBodies.get_value(0, 'articleBody')

Train Bodies (20/1683):


In [29]:
limit = len(trainStances)
limit = 100

In [30]:
print "Train Stances (20/%s):" % (len(trainStances))
# trainStances.head(20)

trainStances["unrelated"] = trainStances["Stance"].apply(lambda x: 1 if x == "unrelated" else 0)
trainStances["agree"] = trainStances["Stance"].apply(lambda x: 1 if x == "agree" else 0)
trainStances["disagree"] = trainStances["Stance"].apply(lambda x: 1 if x == "disagree" else 0)
trainStances["discuss"] = trainStances["Stance"].apply(lambda x: 1 if x == "discuss" else 0)
trainStances["Body"] = [""] * len(trainStances)
for index, row in trainStances[0:limit].iterrows():
    trainStances.loc[index, "Body"] = trainBodies[trainBodies['Body ID'] == row["Body ID"]]['articleBody'].item()
trainStances = trainStances.drop("Body ID", 1)
trainStances = trainStances.drop("Stance", 1)
    
trainStances.head()

Train Stances (20/49972):


,Headline,unrelated,agree,disagree,discuss,Body
0,Police find mass graves with at least '15 bodi...,1,0,0,0,Danny Boyle is directing the untitled film\n\n...
1,Hundreds of Palestinians flee floods in Gaza a...,0,1,0,0,Hundreds of Palestinians were evacuated from t...
2,"Christian Bale passes on role of Steve Jobs, a...",1,0,0,0,30-year-old Moscow resident was hospitalized w...
3,HBO and Apple in Talks for $15/Month Apple TV ...,1,0,0,0,(Reuters) - A Canadian soldier was shot at the...
4,Spider burrowed through tourist's stomach and ...,0,0,1,0,"Fear not arachnophobes, the story of Bunbury's..."


In [31]:
print "Train Stances Random (20/%s)" % (len(trainStancesRandom))
# trainStancesRandom.head(20)

Train Stances Random (20/49972)


In [33]:
# build feature vocabulary
ngram = 1
vectorizer = CountVectorizer(ngram_range=(1, ngram))
trainStances["Number Of Common Words"] = [-1 for x in range(len(trainStances))]

for index, row in trainStances[0:limit].iterrows():
    headline = row['Headline']
    body = row['Body']
    
#     print index
#     print "HEADLINE: \n%s\n" % headline
#     print "BODY: \n%s\n" % body

    headline_words = headline.split()
    body_words = body.split()
    
    num_words_in_common = len(set(headline_words).intersection(body_words))
    
    trainStances.loc[index, "Number Of Common Words"] = num_words_in_common 

trainStances.head()

# I'd like to graph the num_words_in_common integer to the stance classification, to see what kind of correlation we're working with here.

,Headline,unrelated,agree,disagree,discuss,Body,Number Of Common Words
0,Police find mass graves with at least '15 bodi...,1,0,0,0,Danny Boyle is directing the untitled film\n\n...,2
1,Hundreds of Palestinians flee floods in Gaza a...,0,1,0,0,Hundreds of Palestinians were evacuated from t...,10
2,"Christian Bale passes on role of Steve Jobs, a...",1,0,0,0,30-year-old Moscow resident was hospitalized w...,4
3,HBO and Apple in Talks for $15/Month Apple TV ...,1,0,0,0,(Reuters) - A Canadian soldier was shot at the...,2
4,Spider burrowed through tourist's stomach and ...,0,0,1,0,"Fear not arachnophobes, the story of Bunbury's...",6


In [36]:
trainStances[0:limit].corr()

,unrelated,agree,disagree,discuss,Number Of Common Words
unrelated,1.000000,-0.604842,-0.275172,-0.604842,-0.646779
agree,-0.604842,1.000000,-0.067981,-0.149425,0.474475
disagree,-0.275172,-0.067981,1.000000,-0.067981,0.280491
discuss,-0.604842,-0.149425,-0.067981,1.000000,0.255924
Number Of Common Words,-0.646779,0.474475,0.280491,0.255924,1.000000
